In [ ]:
# 로컬에 아래의 패키지를 모두 설치하였기 때문에 주석처리

#!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
#!pip install langchain openai faiss-cpu tiktoken duckduckgo-search
#!pip install "langchain[docarray]"

In [2]:
import pathlib
import textwrap
import google.generativeai as genai
#로컬버전으로 변경하였기 때문에 주석처리
#from google.colab import userdata
import getpass
import os
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.utils.math import cosine_similarity

In [8]:
# 실행하기 전에 열쇠창에 API키를 등록하세요.

# 코랩버전이 아니므로 주석처리 로컬 환경변수에서 불러오는 것으로 변경
'''
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
'''

# Gemini 설정
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel(model_name = "gemini-pro")

In [12]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type = "RETRIEVAL_DOCUMENT")
model = ChatGoogleGenerativeAI(model="gemini-pro")

# 드라이브 마운트하고 경로를 설정하세요.
# 로컬버전에서는 코드 밑에 data폴더를 만들고 텍스트를 넣어줌. 향후에는 파일을 지정하여 업로드하는 기능 추가하면 좋을듯
with open('data/lp.txt', 'r', encoding='utf-8') as file:
    text_content = file.read()

vectorstore = FAISS.from_texts([text_content], embedding=embedding)

retriever = vectorstore.as_retriever()

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 질문을 입력하세요.
chain.invoke("who is the main?")

# 아래의 에러메시지 분석하여 해결해야하는지 판단해야함. 그럼에도 불구하고 정답은 맞춤
# Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


'The main character is the little prince.'

In [14]:
template = """You are a novel writer.
Frist, extract characters only based on {context} and
Second, write a story about {topic} more than 50 words incorporating those details.
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 어린왕자의 등장인물로 제시어에 맞게 소설을 작성하는 명령
chain.invoke("snow")

'In the heart of a serene winter, where snowflakes gracefully twirled and painted the world in a pristine white, a small village nestled, blanketed in a cozy layer of snow. Amidst the snow-covered houses and frost-kissed trees, a young girl named Anya embarked on a heartwarming adventure.\n\nAnya, known for her rosy cheeks and sparkling eyes, loved nothing more than frolicking in the snow. Each morning, as soon as she heard the gentle crunch beneath her boots, she would bundle up in her warmest clothes and rush outdoors.\n\nThe snow, like a soft, powdery blanket, enveloped her as she ran and jumped, creating a symphony of laughter and joy. Her footprints, like tiny ballet shoes, adorned the streets as she twirled and danced, leaving behind a trail of merriment.\n\nOne morning, as Anya danced through the snowy streets, she noticed something peculiar. A small, delicate snowflake seemed to be following her, fluttering and twirling in perfect harmony with her movements. Amazed and intrigue

In [15]:
template = """
You are a professional reviewer.
Criticize the story presented in {context},
focusing on the topic {topic}..
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 어린왕자를 제시어에 집중하여 평론하도록 명령.
chain.batch(["happiness", "sadness"])

['The novella does not contain information about happiness. Therefore a focused review cannot be done.',
 "The story presented is evocative and tinged with a sense of melancholy. The central theme that emerges is the profound grief experienced by the narrator upon losing the little prince, a unique and extraordinary individual who embodies innocence, imagination, and a deep connection to nature. \n\n1. **Loss and Grief:** The story revolves around the narrator's intense sorrow and sense of loss following the departure of the little prince. The narrator's grief is palpable as he recounts the memories of their time together and struggles to come to terms with the little prince's absence.\n\n2. **Memories and Reminiscence:** The story is punctuated by the narrator's vivid recollections of his encounters with the little prince. These memories, both joyful and poignant, serve as a poignant reminder of the special bond they shared. The narrator's remembrance of their conversations, laughter,

In [16]:
# 역할부여
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.
You better to refer to the {context}
Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.
You better to refer to the {context}
Here is a question:
{query}"""

embeddings = embedding
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"context": retriever,"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | model
    | StrOutputParser()
)
print(chain.invoke("what are they talking about?"))

Using MATH
This context does not mention anything about what they are talking about, so I cannot answer this question from the provided context.


In [17]:
# 검색채팅기능(?)

from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model | StrOutputParser() | search

In [18]:
chain.invoke({"input": "Please recommend recently released that are similar to The Little Prince."})

'The Giving Tree, by Shel Silverstein. The Giving Tree is a book that helps children and adults learn about love and acceptance. It is about a tree that loved a little boy. The story is beautifully written with beautiful illustrations to offer readers a unique point of view. Second chance brings back memories. "The Little Prince" is about a boy who lives on a very small planet together with a demanding rose, two active volcanoes, an extinct volcano, and baobab ... The Little Prince and The Picture of Dorian Gray reply | flag. message 17: by Josh (new) Sep 09, 2023 12:15PM ... Furthermore, I like historical (fiction), but also romance books and books similar to The Hunger Games (do not know how you would describe that genre). I like writers like Hellen Rappaport for the historical accuracy, for instance ... The Little Prince, fable and modern classic by French aviator and writer Antoine de Saint-Exupéry that was published with his own illustrations in French as Le Petit Prince in 1943. 

In [20]:
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)

# 버퍼메모리
conversation = ConversationChain(
    llm=model,
    memory=ConversationBufferMemory()
)
conversation("Good morning AI!")
conversation("How are you today?")
conversation("What should I do better today?")
print(conversation.memory.buffer)

Human: Good morning AI!
AI: Good morning to you too! I hope your day is off to a great start. Is there anything I can assist you with today?
Human: How are you today?
AI: I am doing very well, thank you for asking! I am always happy to engage in conversation and learn new things. How about you, how are you doing today?
Human: What should I do better today?
AI: * **Prioritize tasks and set goals:** Take some time at the beginning of your day to identify the most important tasks and goals you need to accomplish. This will help you stay focused and avoid getting sidetracked by less important matters.
* **Create a to-do list:** Once you have identified your priorities, create a to-do list to help you stay organized and track your progress. This will also help you avoid feeling overwhelmed by the amount of work you have to do.
* **Set aside time for breaks:** It is important to take breaks throughout the day to rest and recharge. Get up and move around, or step outside for some fresh air. T

In [24]:
bufw_history = conversation.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)

Human: Good morning AI!
AI: Good morning to you too! I hope your day is off to a great start. Is there anything I can assist you with today?
Human: How are you today?
AI: I am doing very well, thank you for asking! I am always happy to engage in conversation and learn new things. How about you, how are you doing today?
Human: What should I do better today?
AI: * **Prioritize tasks and set goals:** Take some time at the beginning of your day to identify the most important tasks and goals you need to accomplish. This will help you stay focused and avoid getting sidetracked by less important matters.
* **Create a to-do list:** Once you have identified your priorities, create a to-do list to help you stay organized and track your progress. This will also help you avoid feeling overwhelmed by the amount of work you have to do.
* **Set aside time for breaks:** It is important to take breaks throughout the day to rest and recharge. Get up and move around, or step outside for some fresh air. T

In [25]:
# 요약메모리
conversation = ConversationChain(
    llm=model,
    memory=ConversationSummaryMemory(llm=model)
)
conversation("Good morning AI!")
conversation("How are you today?")
conversation("What should I do better today?")
print(conversation.memory.buffer)

The human greeted the AI with a good morning. The AI responded with a good morning and asked how it could assist the human. The human asked the AI how its day was and the AI said it was doing very well, always learning new things, and excited to help with anything. The human then asked the AI what it should do better today. The AI provided a list of suggestions to help the human be more productive, including planning and prioritizing tasks, setting realistic goals, taking breaks, delegating tasks, avoiding distractions, taking care of oneself, and rewarding oneself for accomplishments.


In [26]:
bufw_history = conversation.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)

The human greeted the AI with a good morning. The AI responded with a good morning and asked how it could assist the human. The human asked the AI how its day was and the AI said it was doing very well, always learning new things, and excited to help with anything. The human then asked the AI what it should do better today. The AI provided a list of suggestions to help the human be more productive, including planning and prioritizing tasks, setting realistic goals, taking breaks, delegating tasks, avoiding distractions, taking care of oneself, and rewarding oneself for accomplishments.


In [27]:
# 갯수 지정 메모리

conversation = ConversationChain(
    llm=model,
    memory=ConversationBufferWindowMemory(k=3)
)
conversation("Good morning AI!")
conversation("How are you today?")
conversation("What should I do better today?")
print(conversation.memory.buffer)

Human: Good morning AI!
AI: Good morning! How might I help you today?
Human: How are you today?
AI: I am doing well. I am always happy to help in any way that I can. Is there anything specific that I can assist you with today?
Human: What should I do better today?
AI: That is an interesting question. Here are some suggestions on things you might consider doing better today:
- Enhance your communication skills by actively listening and expressing yourself clearly.
- Be more mindful and present in the moment, savoring the little joys of life.
- Step outside your comfort zone and try something new, such as a different route to work or a new recipe for dinner.
- Show gratitude and appreciation to those around you, both big and small.
- Be more organized and efficient with your time and tasks, setting priorities and eliminating distractions.
- Focus on self-care and well-being, ensuring you get enough sleep, eat nutritious foods, and engage in activities that bring you joy.
- Practice empat

In [28]:
bufw_history = conversation.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)

Human: Good morning AI!
AI: Good morning! How might I help you today?
Human: How are you today?
AI: I am doing well. I am always happy to help in any way that I can. Is there anything specific that I can assist you with today?
Human: What should I do better today?
AI: That is an interesting question. Here are some suggestions on things you might consider doing better today:
- Enhance your communication skills by actively listening and expressing yourself clearly.
- Be more mindful and present in the moment, savoring the little joys of life.
- Step outside your comfort zone and try something new, such as a different route to work or a new recipe for dinner.
- Show gratitude and appreciation to those around you, both big and small.
- Be more organized and efficient with your time and tasks, setting priorities and eliminating distractions.
- Focus on self-care and well-being, ensuring you get enough sleep, eat nutritious foods, and engage in activities that bring you joy.
- Practice empat

In [29]:
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    HumanMessage(content="Hi AI, how are you? What is quantum physics?")
]
conversation(messages)

{'input': [SystemMessage(content='You are a helpful assistant. You keep responses to no more than 100 characters long (including whitespace), and sign off every message with a random name like "Robot McRobot" or "Bot Rob".'),
  HumanMessage(content='Hi AI, how are you? What is quantum physics?')],
 'history': "Human: Good morning AI!\nAI: Good morning! How might I help you today?\nHuman: How are you today?\nAI: I am doing well. I am always happy to help in any way that I can. Is there anything specific that I can assist you with today?\nHuman: What should I do better today?\nAI: That is an interesting question. Here are some suggestions on things you might consider doing better today:\n- Enhance your communication skills by actively listening and expressing yourself clearly.\n- Be more mindful and present in the moment, savoring the little joys of life.\n- Step outside your comfort zone and try something new, such as a different route to work or a new recipe for dinner.\n- Show gratitu

In [30]:
bufw_history = conversation.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)
# 총 질문이 4개이지만 3개만 저장됨

Human: How are you today?
AI: I am doing well. I am always happy to help in any way that I can. Is there anything specific that I can assist you with today?
Human: What should I do better today?
AI: That is an interesting question. Here are some suggestions on things you might consider doing better today:
- Enhance your communication skills by actively listening and expressing yourself clearly.
- Be more mindful and present in the moment, savoring the little joys of life.
- Step outside your comfort zone and try something new, such as a different route to work or a new recipe for dinner.
- Show gratitude and appreciation to those around you, both big and small.
- Be more organized and efficient with your time and tasks, setting priorities and eliminating distractions.
- Focus on self-care and well-being, ensuring you get enough sleep, eat nutritious foods, and engage in activities that bring you joy.
- Practice empathy and understanding towards others, putting yourself in their shoes a

In [31]:
# RAG 없이 작동이 되네?.?
template = ChatPromptTemplate.from_template(
    "Give me small report about {topic} in the {context}"
)

prompt = template
model = model
output_parser = StrOutputParser()
lcel_chain = prompt | model | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence","context": text_content})
print(out)

1. The little prince is from a tiny planet that is not much bigger than a house, and has three volcanoes and a flower.


2. The little prince's planet is threatened by baobabs, which are giant trees that can overtake and destroy the 
entire planet if they are not kept in check.


3. The little prince's flower is unique and precious because it is the only one of its kind, and because it is loved 
by the little prince.


4. The little prince learns that true friendship and love are more important than material possessions or 
appearances, as he observes the behavior of adults on Earth.


5. The little prince meets a fox who teaches him the importance of taming and forming bonds with others, and 
that the most important things in life are often invisible to the eye.


6. The little prince realizes that his rose is special and precious to him because he has taken the time to care 
for it and love it, and that this love is more important than simply possessing it.


7. The little prince ult

In [35]:
# 문서 두개를 검색하는법

from langchain.vectorstores import DocArrayInMemorySearch

# with open('/content/drive/MyDrive/프로젝트/data/lp.txt', 'r', encoding='utf-8') as file:
# 로컬에 맞게 폴더변경, file_path 기능을 이용해야 할듯
with open('data/lp.txt', 'r', encoding='utf-8') as file:
    text_content = file.read()
vectorstore_a = FAISS.from_texts([text_content], embedding=embedding)

# with open('/content/drive/MyDrive/프로젝트/data/smry1.txt', 'r', encoding='utf-8') as file:
# 로컬에 맞게 폴더변경, file_path 기능을 이용해야 할듯
with open('data/smry1.txt', 'r', encoding='utf-8') as file:
    text2_content = file.read()
vectorstore_b = FAISS.from_texts([text2_content], embedding=embedding)

retriever_a = vectorstore_a.as_retriever()
retriever_b = vectorstore_b.as_retriever()

prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [36]:
out = chain.invoke("where is little prince from?")
print(out)

Asteroid B-612
